# COVID-19 Data Visualizations
by Matt Luck (matthew.luck@gmail.com)

In [1]:
# Data

import os, sys
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import branca.colormap as cm

path = '/Users/matt/git/covid-19/'
output_dir = '/Users/matt/projects/covid-19/output/'

# Average number of days between case and death
lag = 14

### Data sources
# https://github.com/nytimes/covid-19-data
# Read directly from URL
covid = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', parse_dates=['date'])

counts = ['cases', 'deaths']

cases_max = covid['cases'].max()
deaths_max = covid['deaths'].max()

date_latest = covid['date'].max()
data_date = date_latest

covid_latest = covid[covid['date']==date_latest]

covid_list = ['county','state','cases','deaths']

# Generate previous data for time lags/trend analyses
covid_lag = covid[covid['date']==(covid['date'].max() - timedelta(days=lag))]
covid_latest = covid_latest.merge(covid_lag[covid_list],
                                  on=['county','state'],
                                  suffixes=('','_{}d'.format(lag)))

"""
covid_1d = covid[covid['date']==(covid['date'].max() - timedelta(days=1))]
covid_2d = covid[covid['date']==(covid['date'].max() - timedelta(days=2))]
covid_7d = covid[covid['date']==(covid['date'].max() - timedelta(days=7))]
covid_8d = covid[covid['date']==(covid['date'].max() - timedelta(days=8))]
covid_9d = covid[covid['date']==(covid['date'].max() - timedelta(days=9))]

covid_latest = covid_latest.merge(covid_1d[covid_list],
                                  on=['county','state'],
                                  suffixes=('','_1d'))
covid_latest = covid_latest.merge(covid_2d[covid_list],
                                  on=['county','state'],
                                  suffixes=('','_2d'))
covid_latest = covid_latest.merge(covid_7d[covid_list],
                                  on=['county','state'],
                                  suffixes=('','_7d'))
covid_latest = covid_latest.merge(covid_8d[covid_list],
                                  on=['county','state'],
                                  suffixes=('','_8d'))
covid_latest = covid_latest.merge(covid_9d[covid_list],
                                  on=['county','state'],
                                  suffixes=('','_9d'))

for p in counts:
    covid_latest['{}_trend_7d'.format(p)] = ((covid_latest[p] + covid_latest['{}_1d'.format(p)] + covid_latest['{}_2d'.format(p)])/3 - (covid_latest['{}_7d'.format(p)] + covid_latest['{}_8d'.format(p)] + covid_latest['{}_9d'.format(p)])/3)
    covid_latest['{}_trend_7d_pop'.format(p)] = covid_latest['{}_trend_7d'.format(p)]/covid_latest[p]*100000
"""


# Fix New York City/County issues
nyc_fips = [36061,36005,36047,36081,36085]
covid_latest = covid_latest[~covid_latest['fips'].isin(nyc_fips)]
covid_latest.loc[covid_latest['county']=='New York City','fips'] = 36061


# https://github.com/CSSEGISandData/COVID-19
population = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv')

# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
counties_data = os.path.join(output_dir, 'counties.geojson')

if os.path.exists(counties_data):
    counties = gpd.read_file(counties_data, driver='GeoJSON')
else:
    counties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip')

    counties['fips'] = counties.GEOID.astype('float')

    counties = counties.merge(population, left_on='fips', right_on='FIPS')

    counties = counties[['fips', 'NAME', 'Population', 'geometry']]

    # Fix New York City/County issues
    counties.loc[counties['fips'].isin(nyc_fips),'fips'] = 36061
    counties = counties.dissolve(by='fips', aggfunc='sum', as_index=False)

    counties.to_file(os.path.join(output_dir, 'counties.geojson'), driver='GeoJSON')

gdf = counties.merge(covid_latest, on='fips')

for p in counts:
    gdf['{}_pop'.format(p)] = gdf[p] / gdf['Population'] * 100000.

gdf['case_fatality'] = np.where(gdf['cases_{}d'.format(lag)]<3, 0, gdf['deaths'] / gdf['cases_{}d'.format(lag)])

del gdf['date']

In [2]:
# Visualization

import folium

def makeMap(param, bins, data_date, fill_color='YlOrBr'):
    m = folium.Map(
        location=[38, -98],
        zoom_start=5,
        min_zoom=4,
        max_zoom=10,
        tiles='cartodbpositron'
    )
    
    tiles = ['cartodbpositron', 'stamenwatercolor', 'openstreetmap', 'stamenterrain']
    for tile in tiles:
        folium.TileLayer(tile).add_to(m)


    c = folium.Choropleth(
        geo_data=gdf,
        data=gdf[['fips', param[0]]],
        name=param[1],
        columns=['fips', param[0]],
        key_on='feature.properties.fips',
        fill_color=fill_color,
        fill_opacity=0.5,
        line_weight=0,
        line_opacity=0.1,
        legend_name='{} as of {}'.format(param[1], data_date.strftime('%b %d, %Y')),
        bins=bins,
        highlight=True,
        reset=True
    )

    # Add tooltip
    t = folium.GeoJsonTooltip(fields=['county',
                                      'Population',
                                      'cases',
                                      'deaths',
                                      'cases_pop',
                                      'deaths_pop',
                                      'case_fatality'],
                              aliases=['County',
                                       'Population',
                                       'Cases',
                                       'Deaths',
                                       'Cases/100000',
                                       'Deaths/100000',
                                       'Case Fatality Rate'],
                             localize=True)

#    g = TimeSliderChoropleth(
#        gdf.set_index("id").to_json(), # get's the coordinates for each id 
#        styledict = styledict # styledict contains for each id the timestamp and the color to plot.
#    )

    t.add_to(c.geojson)
    c.add_to(m)
    
    m.get_root().title = 'COVID-19 maps'
    
    folium.LayerControl(collapsed=True).add_to(m)

    # Save map as web page
    m.save(os.path.join(output_dir, 'covid_{}.html'.format(param[0].lower())))


    # Display map
    #m
    
def makeMultiMap(bins, data_date, fill_color='YlOrBr'):

    m = folium.Map(
        location=[38, -98],
        zoom_start=5,
        min_zoom=4,
        max_zoom=10,
        overlay=False,
        tiles=None
    )

    cases = gdf[['fips','cases']]
    deaths = gdf[['fips','deaths']]
    cases_pop = gdf[['fips','cases_pop']]
    deaths_pop = gdf[['fips','deaths_pop']]

    # feature groups
    feature_group0 = folium.FeatureGroup(name='Cases', overlay=False).add_to(m)
    feature_group1 = folium.FeatureGroup(name='Deaths', overlay=False).add_to(m)
    feature_group2 = folium.FeatureGroup(name='Cases/100000', overlay=False).add_to(m)
    feature_group3 = folium.FeatureGroup(name='Deaths/100000', overlay=False).add_to(m)

    
    fs = [feature_group0, feature_group1, feature_group2, feature_group3]
    counts = ['cases','deaths','cases_pop','deaths_pop']
    count_datas = [cases, deaths, cases_pop, deaths_pop]


    colormap = cm.linear.YlOrRd_09.scale(min(bins), max(bins))
    colormap = colormap.to_step(index=bins, method='log')
    colormap.caption = 'Numbers as of {}'.format(data_date.strftime('%b %d, %Y'))
    colormap.add_to(m)

    for i in range(len(counts)):
        c = folium.Choropleth(
            name=counts[i],
            geo_data=gdf,
            data=count_datas[i],
            columns=['fips', counts[i]],
            key_on='feature.properties.fips',
            fill_color=fill_color,
            #nan_fill_color='gray',
            fill_opacity=0.5,
            line_weight=0,
            line_opacity=0.1,
            legend_name='{} as of {}'.format(param[1], data_date.strftime('%b %d, %Y')),
            bins=bins,
            highlight=True).geojson.add_to(fs[i])


        # Add tooltip
        t = folium.GeoJsonTooltip(fields=['county',
                                          'Population',
                                          'cases',
                                          'deaths',
                                          'cases_pop',
                                          'deaths_pop',
                                          'case_fatality'],
                                  aliases=['County',
                                           'Population',
                                           'Cases',
                                           'Deaths',
                                           'Cases/100000',
                                           'Deaths/100000',
                                           'Case Fatality Rate'],
                                  localize=True)

        t.add_to(c)
    
    m.get_root().title = 'COVID-19 maps by Matt Luck'

    folium.TileLayer(overlay=True, tiles="cartodbpositron", control=False).add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)
    
    # Save map as web page
    m.save(os.path.join(output_dir, 'covid_counts.html'.format(param[0].lower())))
#    m.save('output/covid_{}_{}.html'.format(param[0].lower(), data_date.strftime('%Y-%m-%d')))

In [3]:
# Join COVID cases and deaths to the counties map
"""
params = [('cases', 'Cases'),
          ('case_fatality','Case Fatality Rate = Deaths / (Cases {} days prior)'.format(lag))]
"""
params = [('cases','Cases'),
          ('deaths','Deaths'),
          ('cases_pop','Cases/100000'),
          ('deaths_pop','Deaths/100000'),
          ('case_fatality','Case Fatality Rate = Deaths / (Cases {} days prior)'.format(lag))
          #('cases_trend_7d', '1 Week Trend'),
          #('deaths_trend_7d', '1 Week Trend'),
          #('cases_trend_7d_pop', '1 Week Trend/100000'),
          #('deaths_trend_7d_pop', '1 Week Trend/100000')
         ]
for param in params:
    print(param)
    fill_color = 'YlOrBr'

    if param[0] == 'cases':
        bins = [0.] + [10**x for x in range(np.int(np.floor(np.log10(gdf[param[0]].max())))+1)] + [gdf[param[0]].max()]
        makeMultiMap(bins, date_latest)
    if param[0] in ['cases', 'deaths']:
        bins = [0.] + [10**x for x in range(np.int(np.floor(np.log10(gdf[param[0]].max())))+1)] + [gdf[param[0]].max()]
        makeMap(param, bins, date_latest, fill_color)
    elif param[0] in ['cases_pop', 'deaths_pop']:
        bins = [0.] + [10**x for x in range(np.int(np.floor(np.log10(gdf[param[0]].max())))+1)] + [gdf[param[0]].max()]
        makeMap(param, bins, date_latest, fill_color)

    elif param[0] == 'case_fatality':
        bins = [0, .05, .1, .2, .4, .6, 1]
        makeMap(param, bins, date_latest, fill_color)
    elif param[0] in ['cases_trend_7d', 'deaths_trend_7d', 'cases_trend_7d_pop', 'deaths_trend_7d_pop']:
        bins = [covid_latest[param[0]].min(), covid_latest[param[0]].min()/10., 0, covid_latest[param[0]].max()/10., covid_latest[param[0]].max()]
        #fill_color = 'RdYlBu'
        #colormap = cm.LinearColormap(colors=['lightblue','yellow','red'], index=[int(covid_latest[param[0]].min()/10.),0,int(covid_latest[param[0]].max()/10.)],vmin=int(covid_latest['cases_trend_7d'].min()),vmax=int(covid_latest['cases_trend_7d'].max()))
        makeMap(param, bins, date_latest, fill_color)
#    else:
#        bins = [0., 1e-9, 1e-8, 1e-7, 1e-6, 1]
#        makeMap(param, bins, date_latest, fill_color)

('cases', 'Cases')
('deaths', 'Deaths')
('cases_pop', 'Cases/100000')
('deaths_pop', 'Deaths/100000')
('case_fatality', 'Case Fatality Rate = Deaths / (Cases 14 days prior)')


In [4]:
gdf[gdf['county']=='New York City'].head()

,fips,Population,ALAND,AWATER,geometry,county,state,cases,deaths,cases_14d,deaths_14d,cases_pop,deaths_pop,case_fatality
1643,36061.0,15044928.0,777967926,434656806,"MULTIPOLYGON (((-74.25609 40.50790, -74.25356 ...",New York City,New York,187157,19210,155081,15754,1243.987342,127.684227,0.123871
